In [1]:
import pandas as pd
import xml
import xml.etree.ElementTree as et
import math
import numpy as np
import requests
from datetime import datetime as dt
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy
arcpy.GetInstallInfo()['Version']
gis = GIS("home")

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:597: UserWarning: You are logged on as ben10453@esri.com_DBSNE with an administrator role, proceed with caution.
  warnings.warn(


In [2]:
#Create Region variables and data variables for FAA API Query
region = 'AEA'

case = 'OE'
#case = 'NRA'

year = datetime.datetime.today().year
years = [year - i for i in range(6)]


In [3]:
def parse_XML(xml_string, df_cols): 
    """Parse the input XML string and store the result in a pandas 
    DataFrame with the given columns. 
    """
    
    xroot = et.fromstring(xml_string)
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
    out_df.drop('OECase', axis=1, inplace=True)
    out_df['latitude'] = out_df['latitude'].astype('float64')
    out_df['longitude'] = out_df['longitude'].astype('float64')
    out_df['aglStructureHeight'] = out_df['aglStructureHeight'].astype('int')
    out_df['dateEntered'] = pd.to_datetime(out_df['dateEntered'],format='%Y-%m-%d')
    out_df['expirationDate'] = pd.to_datetime(out_df['expirationDate'],format='%Y-%m-%d')
    out_df['dateCompleted'] = pd.to_datetime(out_df['dateCompleted'],format='%Y-%m-%d')
        
    return out_df

In [4]:
#List of attribute header names matching exact XML schema

headers=["OECase","id",
"asn",
"asnSequence",
"dateEntered",
"expirationDate",
"dateCompleted",
"caseType",
"nearestAirportName",
"nearestState",
"nearestCity",
"siteElevationProposed",
"aglStructureHeight",
"statusCode",
"year",
"sponsor",
"sponsorAddress1",
"sponsorCity",
"sponsorState",
"sponsorPostalCode",
"faaGeographyId",
"distanceFromNearestAirport",
"directionFromNearestAirport",
"structureDescription",
"structureType",
"locatorId",
"latitude",
"longitude",
"sponsorEmail",
"sponsorCountry",
"sponsorPhone",
"receivedDate",
"amslOverallHeightProposed",
"latLongAccuracy",
"EditFlag"]

In [5]:
#Get the XML response from FAA API
df_list = []
for y in years:
    url = f'https://oeaaa.faa.gov/oeaaa/services/caseList/{case}/{y}?region={region}'
    get = requests.get(url)
    content = get.content
    xmlData = content.decode('utf-8')
    out_df=parse_XML(xmlData, headers)
    codes=['JFK','5I6','LGA','NY07','EWR','UNU','SWF','17V','TEB']
    df=out_df[out_df.nearestAirportName.isin(codes)]
    df_list.append(df)
    print(y, "has been appended")



In [6]:
obst_df=pd.concat(df_list)

In [7]:
#Create spatial data frame

sedf= pd.DataFrame.spatial.from_xy(obst_df, 'longitude', 'latitude')


In [ ]:
#Run this if no hosted feature service exists yet
#lyr = sedf.spatial.to_featurelayer('FAA Obstacles')
#lyr

In [8]:
#Get hosted feature layer and create a spatial data frame

item = gis.content.get("4626b7b4c72d417997d0a4da0c0a7ba5")
flayer = item.layers[0]
sdf = pd.DataFrame.spatial.from_layer(flayer)

In [9]:
#Query features

feats = flayer.query()

#Loop through and change Edit Flag

feats_to_edit = []
for feat in feats.features:
    feat_edit = feat # nested dictionary


    old_value = feat.attributes['EditFlag']
    
    new_value = "0"
    
    # update the nested dictionary
    feat_edit.attributes['EditFlag'] = new_value

    feats_to_edit.append(feat_edit)

In [10]:
#Update layer so Edit Flag is ready for new/changed features

max_feats = 500
new_features = feats_to_edit
output_flayer = flayer
if len(new_features) > max_feats:
    # update in chunks
    chunks = math.ceil(len(new_features) / max_feats)
    new_feat_chunks = np.array_split(new_features, chunks)
    print("Adding new features in {} chunks".format(chunks))
    for x in new_feat_chunks:
        output_flayer.edit_features(updates = x)
else:
    output_flayer.edit_features(updates = new_features)



Adding new features in 9 chunks


In [11]:
#Merge function to search on specific field value

def clean_merge(df,value_field=None):

    ## Make a copy of the input data
    df_copy = df.copy(True)
    if value_field is not None:
        df_copy = df_copy.loc[df_copy[value_field+"_old"] != df_copy[value_field]]
    cols = [c for c in df_copy.columns if c.endswith('_old')]
    cols.append('_merge')
    return df_copy.drop(columns=cols)

In [12]:
#Identify function that uses previous merge function to create an add, updates, and deletes df based on merge results.

def identify_edits(exist_df,newdata_df,id_field,value_field=None):
    
    if exist_df.empty:
        adds = newdata_df
        upds = pd.DataFrame()
    else:
        mdf = pd.merge(left=exist_df,
                       right=newdata_df,
                       on=id_field,
                       indicator=True,
                       how="outer",
                       suffixes=['_old','']
                      )
        adds = clean_merge(mdf.loc[mdf['_merge'] == 'right_only'],value_field)
        upds = clean_merge(mdf.loc[mdf['_merge'] == 'both'],value_field)
        dels = clean_merge(mdf.loc[mdf['_merge'] == 'left_only'],value_field)
    ## Fix OBJECTID field
    adds = adds.drop(columns='OBJECTID')
    adds['EditFlag'] = '1'
    upds['OBJECTID'] = upds['OBJECTID'].astype(int)
    upds['EditFlag'] = '1'
    dels['OBJECTID'] = dels['OBJECTID'].astype(int)
    return adds, upds, dels

In [13]:
#Run function

adds,upds,dels = identify_edits(sdf,sedf,"id",value_field="statusCode")

In [14]:
#Add, Update, or Delete features
if adds.empty:
    pass
else:
    add_result = flayer.edit_features(adds = adds.spatial.to_featureset())
    

In [15]:
#Add, Update, or Delete features
if upds.empty:
    pass
else:
    update_result = flayer.edit_features(updates = upds.spatial.to_featureset())
    

In [16]:
#Add, Update, or Delete features
if dels.empty:
    pass
else:
    dels_list = dels["OBJECTID"].tolist()
    delete_result = flayer.edit_features(deletes = dels_list)